# Logistic Regression From Scratch 
--- 

This is a full on implementation of the classification algorithm **Logistic Regression** from scratch using only **Numpy**, including :
- Logistic Function
- Gradient Descent
  - Full Batch
  - Mini-Batch
  - Stocastic
- Cross entropy Loss (Cost function)
- Prediction and accuracy
- Ridge Logistic Regression **L2**
- Lasso Logistic Regression **L1**
- Cross Validation

There will be another jupyter notebook for the **Heart Disease** data set (mini study) + **sklearn** benchmark

## Implementation 

In [1]:
import numpy as np

- We only need **numPy** to implement every logistic regression functionality in this project
- The class implementation based can be found in the `Logistic_Regression_Scratch.py` File which contain better code practices similar to `sklearn` library 

---
### Generating Data

In [40]:
def generate_dummy_data(n=1000,p=3,seed=12):
    np.random.seed(seed)

    X= np.random.randn(n,p)

    coeff_true = np.array([2]*p)
    intercept = 3

    P_x = np.exp(X @ coeff_true + intercept)/(1+ np.exp(X@coeff_true + intercept))
    
    y = np.random.binomial(1,P_x)

    return X,coeff_true, intercept , y

- This function serves as a way to generate dummy data to set our function
- `n` number of **observations** and `p` number of **features** or predictors
- Binary Logistic Regression follow's a **Binomial** distribution so the **true responses** are randomly drawn from it with a probability $P(x)$

$$  p(X)=\frac{e^{\beta_{0}+X\beta}}{1+e^{\beta_{0}+X\beta}}=\frac{1}{e^{-(\beta+X\beta)}+1}$$

- This is called the **Logistic Function** which gives us results between $0$ and $1$ also called the sigmoid function
- $P(x)$ gives us probability results for each observation 

---
### Logistic Function

In [3]:
def sigmoid_function(X,intercept,beta):
    logit = X @ beta + intercept

    P_x = np.where(
        logit >= 0,
        1 / (1 + np.exp(-logit)),
        np.exp(logit) / (1 + np.exp(logit))
    )
    return P_x

- This `sigmoid_function` calculated the probability of an observation as stated above
- The `logit` is just log of the **odds**
$$ odds =p(X)/(1-p(X))$$

- The`logit` is just a **Linear Regression** equation which allow us to do **inference** and statistical analysis on **Logistic Regression** 

---
### Cross Entropy Loss (Cost Function)

In [4]:
def cross_entropy_loss(y,X,beta,intercept):
    eps = 1e-15
    sigmoid_fn = sigmoid_function(X,intercept,beta)
    sigmoid_fn = np.clip(sigmoid_fn, eps, 1 - eps)
    
    cost_fn = -np.mean(y.T*np.log(sigmoid_fn)+(1-y).T*np.log(1-sigmoid_fn))
    return cost_fn

- The cost function for the **Logistic Regression** is called **cross_entropy_loss** given by : $$\mathcal{l}(\beta)=-[y^T \log(p(X))+(1-y)^T\log(1-p(X))] $$

- it's simply the log likelihood of the **maximum likelihood** function
- The **ML** is similar to the binomial distribution **PMF**
- The `-` on the equation is simply for optimizaiton purpose to apply the **Gradient Descent** <br>
(more information and details on the documentation pdf)

---
### Gradient Descent

- Time to fit our logistic regression and estimate the coefficients $beta$, This function will apply all 3 types of known gradient descent 

In [5]:
def gradient_descent(lr,n_itr,batch_size,Y,X,n):
    p = X.shape[1]
    beta_est = np.zeros((p,1))
    intercept_est = 0

    for i in range(n_itr):
        idx = np.random.choice(n,size = batch_size , replace = False)
        X_GD = X[idx]
        Y_GD = Y[idx].reshape(-1,1)

        sigmoid_fn = sigmoid_function(X_GD,intercept_est,beta_est)

        gradient_cel = (1/batch_size)*(X_GD.T@(sigmoid_fn-Y_GD))

        gradient_intercept = (1/batch_size)*np.sum(sigmoid_fn-Y_GD)

        beta_est = beta_est - (lr*gradient_cel)
        intercept_est = intercept_est - (lr*gradient_intercept)

    return beta_est , intercept_est

- The **Logistic Regression** has no closed form solution unlike the **Linear Regression** OLS, so gradient descent is the only way to estimate the coefficients of the model, the gradient of the cost function (cross entropy loss) is : $$ \nabla J(\beta)=\frac{1}{n}X^T(\sigma(X\beta)-y)$$

And for the intercept it's : $$ \nabla J(\beta_{0})=\frac{1}{n}\sum(\sigma(X\beta)-y)$$

- They are simply the pratial derivaiton with respect to $\beta$ and for the intercept for $\beta_{0}$

- The `idx` is simply randomly samples take batches from the data `n`
- Both of `X_GD` and `Y_GD` are samples to used to calculate the gradient for the next step in the **Gradient Descent** algorithm 

---
### Prediction & Accuracy 

In [6]:
def predict(X,intercept,beta,threshold=0.5):
    predicted_probability = sigmoid_function(X,intercept,beta)

    predicted_class = (predicted_probability>= threshold).astype(int)
    return predicted_class, predicted_probability

- This function simply calculate the probability of each observation using the estimated coefficients from the `gradient_descent` function
- Classify based on a `threshold` usually set to $0.5$ to either $0$ or $1$

In [7]:
def accuracy(Y,Y_pred):
    return np.mean(Y_pred==Y)

- comparing the true values of the response `Y` and the predicted values of `Y_pred`
- This will come in handy when we compare different regularizations and hyperparameters

---
### Ridge Logistic Regression (L2)

The same concept as in ridge regression, it **shrinks** the coefficients to prevent overfitting and the model being senstive by introducing some **bias** on the training phase for better **variance** on the predictions, which improve prediction accuracy and reduce colinearity 

In [8]:
def gradient_descent(penalty,lr,n_itr,batch_size,pen,Y,X,n):
    p = X.shape[1]
    beta_est = np.zeros((p,1))
    intercept_est = 0

    for i in range(n_itr):
        idx = np.random.choice(n,size = batch_size , replace = False)
        X_GD = X[idx]
        Y_GD = Y[idx].reshape(-1,1)

        sigmoid_fn = sigmoid_function(X_GD,intercept_est,beta_est)
        if penalty.lower() == "l2":
            penalty_term =pen* beta_est
            gradient_cel = (1/batch_size)*((X_GD.T@(sigmoid_fn-Y_GD))+penalty_term)
        else:
            gradient_cel = (1/batch_size)*(X_GD.T@(sigmoid_fn-Y_GD))
            
        gradient_intercept = (1/batch_size)*np.sum(sigmoid_fn-Y_GD)

        beta_est = beta_est - (lr*gradient_cel)
        intercept_est = intercept_est - (lr*gradient_intercept)

    return beta_est , intercept_est

The ridge logistic regression is simply the cross entropy loss function plus the **penalty term** also known as **regularization term** :
$$-l_{ridge}(\beta,\lambda)=l(\beta)-\frac{\lambda}{2}\lVert \beta \rVert_{2}^2 $$
- $l(\beta)$ is the cost function
- $\lambda$ is the hyperprameter, it's selected using the **Cross Validaiton** method

The gradient of the **ridge logistic regression cost function** is : $$ \nabla J(\beta;\lambda)=-\frac{1}{n}X^\intercal(y-\sigma(X\beta))+\lambda \beta $$

**Note** : The regulzarization is only applied to the coefficients and not the intercept(bias), cause the ridge regression introduce **bias** to the ceofficients and we dont want our baseline $\beta_{0}$ to be bais and most of the time it doesnt cause overfitting or get corelated with other coefficients $\beta$

--- 
### Lasso Logistic Regression (L1)

The **Lasso Logistic Regression** is yet similar to the **Lasso Regression** which uses the $L_{1}$ Norm as a penalty term which is the **absolute value** of the coefficients $\beta$, the lasso logistic regression cost function is given : $$  l_{ridge}(\beta,\lambda)=l(\beta)-{\lambda}\lVert \beta \rVert_{1} $$

With : $${\lambda}\lVert \beta \rVert_{1} = \lambda \sum_{j=1}^{p} |\beta_{j}| $$

- The special thing that the lasso offers is **Sparsity** due to the $L_{1}$ it set some of the coeffcients to **Zero**, which make it useful for **Feature** selection and promoting sparse and interpretble models with less complexity


The gradient of the **Lasso logistic regression** cost function is :$$ \nabla J(\beta;\lambda)=-\frac{1}{n}X^\intercal(y-\sigma(X\beta))+\lambda \text{Sign }(\beta)$$

- The **absolute value** doesn't have a solution at zero, so we use **Subgradient** and **KKT** stationarity condition to prove that it set some coefficients to zero which promote sparsity in the model
- The sub-gradient of $\partial \lVert \beta \rVert_{1} = \begin{cases}
\text{sign}(\hat{\beta_{j}}) &  \quad  , \hat{\beta}_{j} \neq 0 \\
\in[-1,1] & \quad ,\hat{\beta}_{j}=0
\end{cases}$

**Intuition** : 
- there exist many slops for the point zero and if we draw them all we will get the interval $[-1,1]$ when the coefficient $\beta =0$

In [9]:
def gradient_descent(penalty,lr,n_itr,batch_size,pen,Y,X):
    p = X.shape[1]
    beta_est = np.zeros((p,1))
    intercept_est = 0
    n = X.shape[0]

    for i in range(n_itr):
        idx = np.random.choice(n,size = batch_size , replace = False)
        X_GD = X[idx]
        Y_GD = Y[idx].reshape(-1,1)

        sigmoid_fn = sigmoid_function(X_GD,intercept_est,beta_est)
        if penalty.lower() == "l2":
            penalty_term =pen* beta_est
            gradient_cel = (1/batch_size)*((X_GD.T@(sigmoid_fn-Y_GD))+penalty_term)
        elif penalty.lower() == "l1":
            penalty_term = pen * np.sign(beta_est)
            gradient_cel = (1/batch_size)*((X_GD.T@(sigmoid_fn- Y_GD))+penalty_term)
        else:
            gradient_cel = (1/batch_size)*(X_GD.T@(sigmoid_fn-Y_GD))
            
        gradient_intercept = (1/batch_size)*np.sum(sigmoid_fn-Y_GD)

        beta_est = beta_est - (lr*gradient_cel)
        intercept_est = intercept_est - (lr*gradient_intercept)

    return beta_est , intercept_est

- The `np.sign` returns $1$ if `beta_est` is positive
  - $-1$ if `beta_est` is negative
  - $0$ if `beta_est` is zero 

---
### Cross-Validation

**Cross-Validaiton** plays an important role, Since it estimates the **test error rate** which helps us in :
- Model Selection
- Detect Overfitting
- Hyperparameter Tunning ($\lambda$ for the regularization)
- Assessing Model Stability and Detect unsual behavoirs

In [23]:
def K_folds(n,K):
    n = np.arange(n)
    folds = []
    fold_size= len(n)//K
    #print("this is fold size " , fold_size)
    for i in range(K):
        start = i*fold_size
        end = (i+1)*fold_size if i< K-1 else len(n)
        test_idx = n[start:end]
        train_idx = np.concatenate([n[:start],n[end:]])
        folds.append((train_idx,test_idx))

    return folds

In [28]:
def strat_Kfolds(n,K,y):
    class_idx = {}
    for class_label in np.unique(y):
        class_idx[class_label] =np.where(y==class_label)[0]

    #print(class_idx[1])
    #print(class_idx[0])
    folds =[([],[]) for _ in range(K)]

    for class_label,idx in class_idx.items():
        class_samples = len(idx)
        #print(f"the class {class_label} has {class_samples}")
        fold_size_class = class_samples //K
        #print(f"the class {class_label} has fold size of {fold_size_class}")

        class_fold = []
        for i in range (K):
            start = i* fold_size_class 
            end = (i+1)*fold_size_class if i < K-1 else class_samples
            test_idx_class = idx[start:end]
            train_idx_class = np.concatenate([idx[:start],idx[end:]])
            folds[i][0].extend(train_idx_class)
            folds[i][1].extend(test_idx_class)
    #print(folds[1][0])
    folds = [(np.array(train),np.array(test)) for train , test in folds]

    return folds


**K-Fold Cross Validation** is the most widely used approach, Since it's a generalized case of the **LOOCV** by splitting the data into :
- $K$ Folds
- Train on $\to K-1$
- Validate and test on $\to 1$

Here we calculate `start` and `end` to construct our `test_idx` indcies for the test data and `train_idx` for the training indices:
- The indices are stored in `folds` list which will be used on the `cross_validaition` function


In [34]:
def cross_validation(K,X,Y):
    n = X.shape[0]
    #folds = strat_Kfolds(n,K,Y)
    
    folds = K_folds(n,K)
    losses = []
    for i, (train,test)in enumerate(folds):
        X_train = X[train]
        Y_train = Y[train].reshape(-1,1)
        X_test = X[test]
        Y_test = Y[test].reshape(-1,1)
       # print("this is shape of X and Y train",X_train.shape,Y_train.shape)
       # print("this is shape of X and Y test",X_test.shape,Y_test.shape)
        beta_est , intercept_est = gradient_descent(penalty="None",pen="1",lr=0.001,n_itr=200,batch_size=2,Y=Y_train,X=X_train) 
        loss = cross_entropy_loss(Y_test ,X_test,beta_est,intercept_est)
        losses.append(loss)
    return losses , np.mean(losses) 

The **Cross-Validation** algorithm steps are : 
- Split the data into **K-Folds**
- Fit(Train) the model on the training folds
- Validate by calculating the loss
- Repeat for all $\text{K-Folds}$ and each time on Fold act as **test fold** while the others as training sets $K-1$

In [41]:
X,beta,inter,y = generate_dummy_data()
X.shape

(1000, 3)

In [44]:
losses,avg_loss = cross_validation(1000,X,y)
avg_loss

np.float64(0.6687248223905198)

In [43]:
avg_loss

np.float64(0.6794179989133541)

In [16]:
import inspect 

In [17]:
class model:
    def __init__(self,penalty = None , lr=10):
        self.penalty = penalty
        self.lr = lr 
        self.beta = 0
        self.loss = 1
    def change(self):
       self.beta+= 10 

In [105]:
model_1 = model()

In [114]:
class_1 = model_1.__class__
para=inspect.signature(class_1.__init__).parameters
para

mappingproxy({'self': <Parameter "self">,
              'penalty': <Parameter "penalty=None">,
              'lr': <Parameter "lr=10">})

In [107]:
params={k: getattr(model_1,k)for k in para if k!="self"}

In [108]:
cl1=class_1(**params)

In [112]:

model_1.beta

10

In [113]:
cl1.beta

0

In [111]:
model_1.change()

In [117]:
cl1.change()

In [118]:
cl1.beta

20